In [1]:
from nariflow import start_initializer

start_initializer().initializer('tape')

In [2]:
from nariflow import Variable
from nariflow import optimizer
from nariflow import GradientTape
#from nariflow import calc_gradient
from nariflow import layer
from nariflow.models import Model
from nariflow import functions as f
from nariflow.core import elementary_function as ef
from nariflow.core import shape_function as sf
from nariflow.utils import DataSet
import numpy as np
import pandas as pd

np.set_printoptions(suppress=True)

import tensorflow as tf

import matplotlib.pyplot as plt

import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

from tensorflow.keras.layers.experimental import preprocessing

In [7]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models

In [8]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# 픽셀 값을 0~1 사이로 정규화합니다.
train_images, test_images = train_images / 255.0, test_images / 255.0

In [9]:
train_images = train_images.reshape(train_images.shape[0],
                                     train_images.shape[3],
                                     train_images.shape[1],
                                    train_images.shape[2])

test_images = test_images.reshape(test_images.shape[0],
                                     test_images.shape[3],
                                     test_images.shape[1],
                                    test_images.shape[2])

In [12]:
class Models(Model):
    def __init__(self):
        super().__init__()
        self.l1 = layer.Conv2d(64, (3,3))
        self.l2 = layer.Conv2d(128, (2,2))
        self.l3 = layer.Conv2d(64, (3,3))
        self.predict = layer.Linear(10)
        self.flatten = layer.flatten

    def forward(self, x):
        y = self.l1(x)
        y = f.activation.relu(y)
        y = self.l2(x)
        y = f.activation.relu(y)
        y = self.l3(x)
        y = f.activation.relu(y)
        y = self.flatten(y)
        y = self.predict(y)

        return y

lr = 0.01

model = Models()

optimizers = optimizer.Adam(lr)

optimizers.setup(model)

In [13]:
def train_step(X_train, y_train):
    with GradientTape() as tape:
        y_pred = model(X_train)
        loss = f.loss.softmax_cross_entropy(y_pred, y_train)

    tape.CalcGradient()
    optimizers.update()

    if j % 100 == 0:
        print(loss.data)

In [14]:
iters = 0

batch_size = 20

start_time = time.time()

In [15]:
while True:
    for j in range(np.ceil((len(train_labels) / batch_size)).astype('int')):

        train = train_images[j * batch_size : (j + 1) * batch_size]
        label = train_labels[j * batch_size : (j + 1) * batch_size]

        train_step(train, label)

    print('total_time : ', time.time() - start_time)

    iters += 1

    print(iters)
    
    if iters > 2:
        break

2.30251580637426
0.5288214253562966
0.34446868387937835
0.0667989756155288
0.32421359959645474
0.1465860960784961
0.11628970241194718
0.4226169812726751
0.11648898529861285
0.18312222800262093
0.24632828794954822
0.08888848865689523
0.044010529182516966
0.0332682066224103
0.12984528057700265
0.09506734619613579
0.3663379229244402
0.084257730366334
0.216741140102009
0.02398759264395197
0.10881779914343111
0.1835789179651264
0.2674011474922151
0.2934319981917307
0.3200113553648062
0.0400353579521381
0.09951490193934279
0.019227553479286584
0.5995717247676052
0.015780027487537495
total_time :  698.0289196968079
1
0.0596537013357731
0.14447714398275754
0.12791466353431086
0.014371169644018739
0.08143634423350428
0.0374367591741108
0.14036093828522556
0.2310412989877535
0.041972339711501286
0.16223922196674895
0.1017580178474419
0.025660653222865072
0.03542082140343732
0.025031697806995146
0.06497574855460095
0.05703916422402568
0.2857335030206715
0.08784368805121182
0.16667031246462566
0.0

In [16]:
test_pred = model(test_images)

In [20]:
print(f.loss.softmax(test_pred.data).data[0:100])

[[0.00000006 0.         0.00000006 0.00000074 0.00000001 0.
  0.         0.99999546 0.00000017 0.0000035 ]
 [0.00000158 0.00000263 0.9999712  0.0000001  0.         0.00000008
  0.00000012 0.         0.00002428 0.        ]
 [0.00000177 0.99936458 0.00006772 0.00000146 0.0001174  0.00000538
  0.00000488 0.00002312 0.00041356 0.00000013]
 [0.99808117 0.00000006 0.00053652 0.00002791 0.0000017  0.00003023
  0.00052433 0.0000002  0.00002146 0.00077641]
 [0.00000231 0.         0.00001686 0.00000011 0.96205518 0.
  0.         0.00000895 0.00001147 0.03790511]
 [0.00000186 0.99904566 0.00009017 0.00000835 0.00005993 0.00001134
  0.00000534 0.00011588 0.00066111 0.00000035]
 [0.         0.00023302 0.00003211 0.00010871 0.9446441  0.00000003
  0.00001219 0.00001682 0.05471695 0.00023608]
 [0.00020784 0.00094038 0.00062262 0.12399165 0.04503946 0.09076729
  0.00037764 0.00066523 0.00104336 0.73634454]
 [0.0000753  0.00000001 0.00000755 0.00000372 0.00008311 0.45657324
  0.54221217 0.00000001 0.00

In [18]:
print('test셋 정확도 : ', accuracy_score(np.argmax(test_pred.data, axis = -1), test_labels))

test셋 정확도 :  0.9459


In [19]:
print('혼동행렬')
confusion_matrix(np.argmax(test_pred.data, axis = -1), test_labels)

혼동행렬


array([[ 955,    1,    2,    0,    5,    3,    9,    1,    6,    2],
       [   0, 1118,   12,    0,    1,    0,    4,    3,    1,    6],
       [   2,    3,  984,    1,    3,    0,    1,   20,   11,    0],
       [   1,    3,   12,  979,    1,   24,    1,   15,   24,   15],
       [   0,    2,    2,    0,  941,    2,    9,    2,    8,    5],
       [   7,    3,    0,   15,    1,  848,   72,    2,   21,    7],
       [   3,    2,    0,    0,    3,    2,  854,    0,    2,    0],
       [   1,    0,    8,    4,    3,    2,    2,  960,   11,   27],
       [  10,    3,   12,    8,    7,    7,    6,   11,  888,   15],
       [   1,    0,    0,    3,   17,    4,    0,   14,    2,  932]],
      dtype=int64)